In [29]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

from vectice import connect, S3DataWrapper, models, api, Model, DatasetSourceUsage



ImportError: cannot import name 'connect' from 'vectice' (unknown location)

In [18]:
# Load dataset
url = "https://raw.githubusercontent.com/vectice/GettingStarted/main/22.4/tutorial/Data/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = read_csv(url, names=names)

In [28]:
## Goal of this sample is to simply catalog assets in Vectice

# Connect to Vectice
conn = vectice.connect(
    api_token='gZq7A41al.xxGVXQ0on8JDPA6z2j4r9gZq7A41albEMLdOmvp3kwW7YeRNBy', 
    host='https://app.vectice.com',
    workspace='Samples'
)
# print(conn.projects) # get list of projects
project = conn.project(6738)

AttributeError: module 'vectice' has no attribute 'connect'

In [ ]:
#prj_sdw = DataFrameDataWrapper(df= dataset, name="Iris Dataset",capture_code=False)
#prj_fdw = FileDataWrapper(path= 'iris.csv', name="Iris Dataset",capture_code=False)
#print(prj_sdw)
#project.origin_dataset = prj_fdw

In [ ]:
# Plot the dataset
dataset.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
boxplot = pyplot.savefig("boxplot.jpg")
pyplot.show()

# histograms
dataset.hist()
histogram = pyplot.savefig("histogram.jpg")
pyplot.show()

In [ ]:
# Split-out validation dataset
array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

# Build Vectice objs

# Set the origin dataset for the project
project.origin_dataset = FileDataWrapper(path= 'iris.csv', name="Iris Dataset",capture_code=False)

# This dataset is now available for use as a Dataset Asset Widget in Vectice webapp

training_wrapped = vectice.FileDataWrapper(path="iris.csv", name="iteration 2", usage=vectice.DatasetSourceUsage.TRAINING, capture_code=False)
testing_wrapped = vectice.FileDataWrapper(path="iris2.csv", name="iteration 2", usage=vectice.DatasetSourceUsage.TESTING, capture_code=False)
validation_wrapped = vectice.FileDataWrapper(path="iris3.csv", name="iteration 2", usage=vectice.DatasetSourceUsage.VALIDATION, capture_code=False)

# I can define other datasets that I use on the project - they will become available as widgets
project.phase("Modeling").clean_dataset = training_wrapped

#project.phase("Modeling").training_dataset = training_wrapped
#project.phase("Modeling").testing_dataset = testing_wrapped
#project.phase("Modeling").validation_dataset = validation_wrapped


In [ ]:
# Spot Check Algorithms
models = []
models.append(('Logistic Regression', 'LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('Linear Discriminant Analysis', 'LDA', LinearDiscriminantAnalysis()))
models.append(('KNeighbors Classifier', 'KNN', KNeighborsClassifier()))
models.append(('Decision Tree Classifier','CART',  DecisionTreeClassifier()))
models.append(('Gaussian NB', 'NB', GaussianNB()))
models.append(('Support Vector Classifier','SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []
#iterStep = project.phase("Modeling").iteration.step("Build model")
for name, label, model in models:
 iterStep = project.phase("Modeling").iteration.step("Build model")
 kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}
 cv_results = cross_validate(model, X_train, Y_train, cv=kfold, scoring=scoring)
 results.append(cv_results['test_acc'])
 names.append(label)
 metrics = {"Accuracy Mean": cv_results['test_acc'].mean(), 
            "Accuracy Std.": cv_results['test_acc'].std(), 
            "Score Time Mean": cv_results['score_time'].mean(),
            "Score Time Std.": cv_results['score_time'].std()}
 #Document model in Vectice - 1 line
 #project.phase("Modeling").iterations[0].model = vectice.Model(name= 'Iris Classification Models', library= name, technique=name, metrics=metrics, attachments=['boxplot.jpg','histogram.jpg'], capture_code= True)
 iterStep.model  = vectice.Model(name= 'Iris Classification Models', library= name, technique=name, metrics=metrics, attachments=['boxplot.jpg','histogram.jpg'], capture_code= True)
 iterStep.close(message="Iteration done\n" + str(metrics))
# Close Vectice step
#iterStep.close(message="Iteration done")

In [ ]:
iterStep.close(message="Iteration done")
#print(project.phase("Modeling").iterations[5].properties)

In [ ]:
# Compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.savefig("algComp.jpg")
pyplot.show()